# T2.8 Extraer con selenium datos de licitacións do Concello da Coruña
## Ralphy Núñez Mercado

Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-5-mssql-server/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

Entrega:

O código nun jupyter notebook.
A BBDD exportada de Microsoft SQL Server.
Un zip coas capturas de pantalla que fixo selenium.

### ⬇️ Imports 

In [124]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random

### Definir Funciones

In [125]:
def waitFinishLoad(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "page-loader")))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.ID, "page-loader")))
    alea=random.choice([0.5, 0.1, 0.2, 0.3, 0.4, 0.6])
    time.sleep(alea)

def clickSelectByPartialText(auxText, driver):
    waitFinishLoad(driver)
    driver.find_element_by_xpath("//select//option[contains(., '"+auxText+"')]").click()
    waitFinishLoad(driver)

def getCurrentPage(driver):
    print()

def getLastPage(driver):
    print()

def isThisLastPage(driver):
    if (getLastPage(driver) == getCurrentPage(driver)):
        return True
    return False

def goNextPage(driver,DOMINIO):
    driver.find_element(by=By.ID, value="id_domain").send_keys(DOMINIO)

    elem = driver.find_element(By.ID, "loginBtn")
    elem.click()

def clickByPartialText(auxText, driver):
    waitFinishLoad(driver)
    elem = driver.find_element_by_partial_link_text(auxText)
    elem.click()
    waitFinishLoad(driver)
    
def buscar_by_id(driver,id):
    driver.find_element(By.ID,id).click()

def buscar_tree(driver,paso):
    driver.find_element(By.XPATH, f"//td[@class='multiline'][following-sibling::td[2]/div[text()='{paso}']]").click()


### Definir Variables

In [126]:

URL = "https://contrataciondelestado.es/wps/portal/plataforma"

# lista_tree = ['ENTIDADES LOCALES','Galicia','A Coruña','Ayuntamientos','A Coruña']
lista_tree = ['ENTIDADES LOCALES','Galicia','A Coruña','Ayuntamientos']

In [ ]:
driver = webdriver.Firefox()

driver.get(URL)

driver.find_element(By.XPATH, "//div/p/a[contains(@href,'/wps/portal/licitaciones')]").click() # click en publicaciones


driver.find_element(By.CLASS_NAME,'divLogo').click() # click en licitaciones


buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada') # click en busqueda avanzada

buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink') # click en selecionar (organizacion contratante)


# for e in lista_tree:
#    buscar_tree(driver,e) # clicks hasta ayuntamiento 






driver.close


InvalidSelectorException: Message: Given xpath expression "//td[@class='multiline'][following-sibling::td[2]/div" is invalid: SyntaxError: Document.evaluate: The expression is not a legal expression; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
InvalidSelectorError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:443:5
find_@chrome://remote/content/shared/DOM.sys.mjs:170:11
